# 天地图，使用APIKEY，批量抓取城市经纬度

In [ ]:
import requests
import pandas as pd
import time
import json
from tqdm import tqdm

class TiandituGeocoder:
    def __init__(self, api_key):
        """
        初始化天地图地理编码器
        
        Args:
            api_key: 天地图API密钥
        """
        self.api_key = api_key
        self.base_url = "https://api.tianditu.gov.cn/geocoder"
        
    def get_coordinates(self, city_name):
        """
        获取城市经纬度坐标
        

        Args:
            city_name: 城市名称
            
        Returns:
            tuple: (纬度, 经度) 或 (None, None) 如果失败
        """
        # 在 __init__ 中添加默认 headers（可选）
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
            'Referer': 'https://www.tianditu.gov.cn/'
        }

        # # 在 get_coordinates 发起请求时传入 headers
        # response = requests.get(self.base_url, params=params, headers=self.headers, timeout=10)


        try:
            # 构建请求参数
            params = {
                'ds': json.dumps({'keyWord': city_name}, ensure_ascii=False),
                'tk': self.api_key
            }
            
            # 发送请求
            response = requests.get(self.base_url, params=params, headers=self.headers, timeout=10)
            
            response.raise_for_status()
            
            # 解析响应
            data = response.json()
            
            # 调试信息
            print(f"查询 {city_name}: 状态码={data.get('status')}, 消息={data.get('msg')}")
            
            if data.get('status') == '0' and data.get('location'):
                location = data['location']
                lat = location.get('lat')
                lon = location.get('lon')
                if lat and lon:
                    print(f"成功获取 {city_name}: lat={lat}, lon={lon}")
                    return float(lat), float(lon)
                else:
                    print(f"坐标数据不完整: {data}")
                    return None, None
            else:
                print(f"获取 {city_name} 坐标失败: {data}")
                return None, None
                
        except requests.exceptions.RequestException as e:
            print(f"请求 {city_name} 时出错: {e}")
            return None, None
        except json.JSONDecodeError as e:
            print(f"解析 {city_name} 响应时出错: {e}")
            return None, None
        except Exception as e:
            print(f"处理 {city_name} 时发生未知错误: {e}")
            return None, None

def flatten_city_list(province_dict):
    """
    将省份字典扁平化为城市列表
    
    Args:
        province_dict: 省份城市字典
        
    Returns:
        list: 所有城市的列表
    """
    all_cities = []
    for province, cities in province_dict.items():
        for city in cities:
            all_cities.append(city)
    return all_cities

def batch_geocode(cities, api_key, batch_size=5, delay=2):
    """
    批量获取城市坐标
    
    Args:
        cities: 城市列表
        api_key: API密钥
        batch_size: 每批处理数量
        delay: 批处理间延迟（秒）
        
    Returns:
        list: 包含坐标信息的字典列表
    """
    geocoder = TiandituGeocoder(api_key)
    results = []
    
    print(f"开始处理 {len(cities)} 个城市...")
    
    for i, city in enumerate(tqdm(cities, desc="获取城市坐标")):
        lat, lon = geocoder.get_coordinates(city)
        
        if lat is not None and lon is not None:
            results.append({
                '城市名称': city,
                'lat纬度': lat,
                'lon经度': lon
            })
        else:
            # 如果第一次失败，可以尝试添加"省"信息
            print(f"尝试添加省信息查询 {city}")
            # 这里可以根据需要添加省份信息，但通常城市名足够
            
        # 控制请求频率，避免触发API限制
        if (i + 1) % batch_size == 0 and i < len(cities) - 1:
            print(f"已处理 {i+1} 个城市，等待 {delay} 秒...")
            time.sleep(delay)
    
    return results

def save_to_excel(data, filename="城市坐标数据.xlsx"):
    """
    保存数据到Excel文件
    
    Args:
        data: 数据列表
        filename: 输出文件名
    """
    if not data:
        print("没有数据可保存")
        return
    
    # 创建DataFrame
    df = pd.DataFrame(data)
    
    # 确保列顺序正确
    df = df[['城市名称', 'lat纬度', 'lon经度']]
    
    # 保存到Excel
    df.to_excel(filename, index=False, engine='openpyxl')
    print(f"数据已保存到 {filename}")
    print(f"成功获取 {len(data)} 个城市的坐标")
    
    # 显示前几行数据
    print("\n数据预览:")
    print(df.head())

def main():
    """主函数"""
    # 您的API密钥
    API_KEY = "8fcc85b07ccf55ee84665c36b53f0aa6"
    zhixiashi_list2 = {'zhixiashi': ['邢台市', '阳泉市', '阿拉善盟', '本溪市', '白山市', '常州市', '马鞍山市', '滁州市', '新余市', '泰安市', '洛阳市', '南阳市', '济源市', '黄冈市', '湘潭市', '娄底市', '珠海市', '河源市', '贵港市', '来宾市', '东方市', '琼中黎族苗族自治县', '达州市', '黔南布依族苗族自治州', '林芝市', '陇南市', '铁门关市']}

    # # 省份城市列表（使用您提供的完整列表）
    # province_list = {
    #     '河北省': ['石家庄市', '唐山市', '秦皇岛市', '邯郸市', '邢台市', '保定市', '张家口市', '承德市', '沧州市',
    #                '廊坊市', '衡水市', '雄安新区'],
    #     '山西省': ['太原市', '大同市', '阳泉市', '长治市', '晋城市', '朔州市', '晋中市', '运城市', '忻州市', '临汾市',
    #                '吕梁市'],
    #     '内蒙古自治区': ['呼和浩特市', '包头市', '乌海市', '赤峰市', '通辽市', '鄂尔多斯市', '呼伦贝尔市', '巴彦淖尔市',
    #                      '乌兰察布市', '兴安盟', '锡林郭勒盟', '阿拉善盟'],
    #     '辽宁省': ['沈阳市', '大连市', '鞍山市', '抚顺市', '本溪市', '丹东市', '锦州市', '营口市', '阜新市', '辽阳市',
    #                '盘锦市', '铁岭市', '朝阳市', '葫芦岛市'],
    #     '吉林省': ['长春市', '吉林市', '四平市', '辽源市', '通化市', '白山市', '松原市', '白城市', '延边朝鲜族自治州'],
    #     '黑龙江省': ['哈尔滨市', '齐齐哈尔市', '鸡西市', '鹤岗市', '双鸭山市', '大庆市', '伊春市', '佳木斯市',
    #                  '七台河市', '牡丹江市', '黑河市', '绥化市', '大兴安岭地区'],
    #     '江苏省': ['南京市', '无锡市', '徐州市', '常州市', '苏州市', '南通市', '连云港市', '淮安市', '盐城市', '扬州市',
    #                '镇江市', '泰州市', '宿迁市'],
    #     '浙江省': ['杭州市', '宁波市', '温州市', '嘉兴市', '湖州市', '绍兴市', '金华市', '衢州市', '舟山市', '台州市',
    #                '丽水市'],
    #     '安徽省': ['合肥市', '芜湖市', '蚌埠市', '淮南市', '马鞍山市', '淮北市', '铜陵市', '安庆市', '黄山市', '滁州市',
    #                '阜阳市', '宿州市', '六安市', '亳州市', '池州市', '宣城市'],
    #     '福建省': ['福州市', '厦门市', '莆田市', '三明市', '泉州市', '漳州市', '南平市', '龙岩市', '宁德市'],
    #     '江西省': ['南昌市', '景德镇市', '萍乡市', '九江市', '新余市', '鹰潭市', '赣州市', '吉安市', '宜春市', '抚州市',
    #                '上饶市'],
    #     '山东省': ['济南市', '青岛市', '淄博市', '枣庄市', '东营市', '烟台市', '潍坊市', '济宁市', '泰安市', '威海市',
    #                '日照市', '临沂市', '德州市', '聊城市', '滨州市', '菏泽市'],
    #     '河南省': ['郑州市', '开封市', '洛阳市', '平顶山市', '安阳市', '鹤壁市', '新乡市', '焦作市', '濮阳市', '许昌市',
    #                '漯河市', '三门峡市', '南阳市', '商丘市', '信阳市', '周口市', '驻马店市', '济源市'],
    #     '湖北省': ['武汉市', '黄石市', '十堰市', '宜昌市', '襄阳市', '鄂州市', '荆门市', '孝感市', '荆州市', '黄冈市',
    #                '咸宁市', '随州市', '恩施土家族苗族自治州', '仙桃市', '潜江市', '天门市', '神农架林区'],
    #     '湖南省': ['长沙市', '株洲市', '湘潭市', '衡阳市', '邵阳市', '岳阳市', '常德市', '张家界市', '益阳市', '郴州市',
    #                '永州市', '怀化市', '娄底市', '湘西土家族苗族自治州'],
    #     '广东省': ['广州市', '韶关市', '深圳市', '珠海市', '汕头市', '佛山市', '江门市', '湛江市', '茂名市', '肇庆市',
    #                '惠州市', '梅州市', '汕尾市', '河源市', '阳江市', '清远市', '东莞市', '中山市', '潮州市', '揭阳市',
    #                '云浮市'],
    #     '广西壮族自治区': ['南宁市', '柳州市', '桂林市', '梧州市', '北海市', '防城港市', '钦州市', '贵港市', '玉林市',
    #                        '百色市', '贺州市', '河池市', '来宾市', '崇左市'],
    #     '海南省': ['海口市', '三亚市', '三沙市', '儋州市', '五指山市', '琼海市', '文昌市', '万宁市', '东方市', '定安县',
    #                '屯昌县', '澄迈县', '临高县', '白沙黎族自治县', '昌江黎族自治县', '乐东黎族自治县', '陵水黎族自治县',
    #                '保亭黎族苗族自治县', '琼中黎族苗族自治县'],
    #     '四川省': ['成都市', '自贡市', '攀枝花市', '泸州市', '德阳市', '绵阳市', '广元市', '遂宁市', '内江市', '乐山市',
    #                '南充市', '眉山市', '宜宾市', '广安市', '达州市', '雅安市', '巴中市', '资阳市', '阿坝藏族羌族自治州',
    #                '甘孜藏族自治州', '凉山彝族自治州'],
    #     '贵州省': ['贵阳市', '六盘水市', '遵义市', '安顺市', '毕节市', '铜仁市', '黔西南布依族苗族自治州',
    #                '黔东南苗族侗族自治州', '黔南布依族苗族自治州'],
    #     '云南省': ['昆明市', '曲靖市', '玉溪市', '保山市', '昭通市', '丽江市', '普洱市', '临沧市', '楚雄彝族自治州',
    #                '红河哈尼族彝族自治州', '文山壮族苗族自治州', '西双版纳傣族自治州', '大理白族自治州',
    #                '德宏傣族景颇族自治州', '怒江傈僳族自治州', '迪庆藏族自治州'],
    #     '西藏自治区': ['拉萨市', '日喀则市', '昌都市', '林芝市', '山南市', '那曲市', '阿里地区'],
    #     '陕西省': ['西安市', '铜川市', '宝鸡市', '咸阳市', '渭南市', '延安市', '汉中市', '榆林市', '安康市', '商洛市'],
    #     '甘肃省': ['兰州市', '嘉峪关市', '金昌市', '白银市', '天水市', '武威市', '张掖市', '平凉市', '酒泉市', '庆阳市',
    #                '定西市', '陇南市', '临夏回族自治州', '甘南藏族自治州'],
    #     '青海省': ['西宁市', '海东市', '海北藏族自治州', '黄南藏族自治州', '海南藏族自治州', '果洛藏族自治州',
    #                '玉树藏族自治州', '海西蒙古族藏族自治州'],
    #     '宁夏回族自治区': ['银川市', '石嘴山市', '吴忠市', '固原市', '中卫市'],
    #     '新疆维吾尔自治区': ['乌鲁木齐市', '克拉玛依市', '吐鲁番市', '哈密市', '昌吉回族自治州', '博尔塔拉蒙古自治州',
    #                          '巴音郭楞蒙古自治州', '阿克苏地区', '克孜勒苏柯尔克孜自治州', '喀什地区', '和田地区',
    #                          '伊犁哈萨克自治州', '塔城地区', '阿勒泰地区', '石河子市', '阿拉尔市', '图木舒克市',
    #                          '五家渠市', '北屯市', '铁门关市', '双河市', '可克达拉市', '昆玉市', '胡杨河市', '新星市',
    #                          '白杨市']
    # }
    
    # 扁平化城市列表
    all_cities = flatten_city_list(zhixiashi_list2)
    print(f"总共需要获取 {len(all_cities)} 个城市/地区的坐标")
    
    # 批量获取坐标
    results = batch_geocode(
        cities=all_cities,
        api_key=API_KEY,
        batch_size=5,    # 每批5个城市
        delay=2          # 每批间隔2秒
    )
    
    # 保存到Excel
    save_to_excel(results, "直辖市城市经纬度坐标.xlsx")
    
    # 如果有失败的城市，打印出来
    successful_cities = set([r['城市名称'] for r in results])
    failed_cities = [city for city in all_cities if city not in successful_cities]
    
    if failed_cities:
        print(f"\n以下 {len(failed_cities)} 个城市获取失败:")
        for city in failed_cities:
            print(f"  - {city}")
        
        # 保存失败的城市列表
        with open("失败城市列表.txt", "w", encoding="utf-8") as f:
            for city in failed_cities:
                f.write(f"{city}\n")
        print("失败城市列表已保存到 失败城市列表.txt")

if __name__ == "__main__":

    
    main()

In [ ]:
# 测试单元：对比带/不带请求头的单次请求，检查 API 响应
# 说明：运行此单元以验证你的 API_KEY 是否在脚本中能正常工作。
API_KEY = "8fcc85b07ccf55ee84665c36b53f0aa6"
try:
    API_KEY
except NameError:
    API_KEY = input('请输入你的天地图 API_KEY：').strip()

import json, requests

def single_test(city, use_headers=False):
    url = 'https://api.tianditu.gov.cn/geocoder'
    params = {'ds': json.dumps({'keyWord': city}, ensure_ascii=False), 'tk': API_KEY}
    headers = None
    if use_headers:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
            'Referer': 'https://www.tianditu.gov.cn/'
        }
    try:
        r = requests.get(url, params=params, headers=headers, timeout=15)
        print('\n== 测试:', city, 'use_headers=', use_headers, '==')
        print('请求 URL:', r.request.url)
        print('返回状态码:', r.status_code)
        # 显示响应头短摘
        print('响应头示例:', {k:v for k,v in list(r.headers.items())[:5]})
        text = r.text
        print('响应长度:', len(text))
        try:
            data = r.json()
            print('解析 JSON 成功, status:', data.get('status'))
            print('示例 location:', data.get('location'))
        except Exception as e:
            print('无法解析为 JSON:', e)
            print('响应文本前300字符:\n', text[:300])
    except Exception as e:
        print('请求异常:', e)

# 运行两次测试：不带 headers 与 带 headers
city_to_test = '北京市'
single_test(city_to_test, use_headers=False)
single_test(city_to_test, use_headers=True)

# 如果带 headers 成功而不带失败，说明 API 对程序化请求有访问限制（需要添加合适的 headers 或使用代理/不同密钥）。
# 运行后请把两个请求的完整输出粘贴给我，我会据此给出下一步调整建议。

# 计算城市间距离矩阵

## 城市间距离矩阵计算（Haversine 公式）

使用 haversine 公式计算两个城市间的大圆距离（单位：km）。
- 公式：a = sin²(Δφ/2) + cos(φ1) × cos(φ2) × sin²(Δλ/2)
- c = 2 × atan2(√a, √(1−a))
- d = R × c（R = 6371 km）


In [6]:
import numpy as np
import pandas as pd

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    计算两点间的 Haversine 距离（单位：km）
    
    Args:
        lat1, lon1: 第一个点的纬度、经度（度）
        lat2, lon2: 第二个点的纬度、经度（度）
    
    Returns:
        float: 距离（km）
    """
    R = 6371  # 地球半径，单位：km
    
    # 转换为弧度
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Haversine 公式
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    
    distance = R * c
    return distance

def compute_distance_matrix(excel_file="全国城市经纬度坐标.xlsx", output_file="城市距离矩阵.xlsx"):
    """
    计算城市间距离矩阵并保存为 Excel
    
    Args:
        excel_file: 输入的城市坐标 Excel 文件名
        output_file: 输出的距离矩阵 Excel 文件名
    """
    print(f"读取文件: {excel_file}")
    try:
        # 读取城市坐标数据
        df = pd.read_excel(excel_file)
        print(f"成功读取 {len(df)} 个城市的坐标数据\n")
        
        # 提取坐标
        city_names = df['城市名称'].values
        lats = df['lat纬度'].values
        lons = df['lon经度'].values
        
        # 计算距离矩阵
        n = len(city_names)
        distance_matrix = np.zeros((n, n))
        
        print(f"开始计算 {n}x{n} 的距离矩阵...")
        for i in range(n):
            if (i + 1) % 100 == 0:
                print(f"已处理 {i+1}/{n} 个城市")
            for j in range(i, n):
                dist = haversine_distance(lats[i], lons[i], lats[j], lons[j])
                distance_matrix[i, j] = dist
                distance_matrix[j, i] = dist  # 距离矩阵对称
        
        # 创建 DataFrame，行列都是城市名称
        distance_df = pd.DataFrame(
            distance_matrix,
            index=city_names,
            columns=city_names
        )
        
        # 保存为 Excel
        print(f"\n保存距离矩阵到 {output_file}...")
        distance_df.to_excel(output_file)
        print(f"✓ 距离矩阵已保存\n")
        
        # 显示矩阵的一部分（前 5x5）
        print("距离矩阵预览（前5个城市）：")
        print(distance_df.iloc[:5, :5].round(2))
        print(f"\n矩阵统计：")
        print(f"最小距离（非零）: {distance_matrix[distance_matrix > 0].min():.2f} km")
        print(f"最大距离: {distance_matrix.max():.2f} km")
        print(f"平均距离: {distance_matrix[distance_matrix > 0].mean():.2f} km")
        
        return distance_df
        
    except FileNotFoundError:
        print(f"❌ 文件 {excel_file} 不存在，请确保已运行坐标获取代码")
        return None
    except Exception as e:
        print(f"❌ 出错: {e}")
        return None

# 运行距离矩阵计算
distance_matrix_df = compute_distance_matrix()

读取文件: 全国城市经纬度坐标.xlsx
成功读取 370 个城市的坐标数据

开始计算 370x370 的距离矩阵...
已处理 100/370 个城市
已处理 200/370 个城市
已处理 300/370 个城市

保存距离矩阵到 城市距离矩阵.xlsx...
✓ 距离矩阵已保存

距离矩阵预览（前5个城市）：
          北京市      天津市      上海市      重庆市     石家庄市
北京市      0.00    86.11  1055.20  1476.06   281.69
天津市     86.11     0.00   971.37  1463.68   282.14
上海市   1055.20   971.37     0.00  1441.97   988.74
重庆市   1476.06  1463.68  1441.97     0.00  1194.48
石家庄市   281.69   282.14   988.74  1194.48     0.00

矩阵统计：
最小距离（非零）: 7.28 km
最大距离: 4557.66 km
平均距离: 1503.46 km


## 距离衰减矩阵（EDF）

计算衰减函数 EDF(i,j) = exp(-alpha * dist(i,j))，默认 alpha=0.01，并将结果保存为 Excel。

In [7]:
import numpy as np
import pandas as pd

def compute_decay_matrix(distance_df=None, alpha=0.01, output_file="城市距离衰减矩阵.xlsx"):
    """计算衰减矩阵 EDF = exp(-alpha * dist) 并保存为 Excel。"""
    try:
        if distance_df is None:
            distance_df = pd.read_excel("城市距离矩阵.xlsx", index_col=0)
        print(f"读取距离矩阵：{distance_df.shape[0]} 城市")
        dist_vals = distance_df.values
        edf_vals = np.exp(-alpha * dist_vals)
        edf_df = pd.DataFrame(edf_vals, index=distance_df.index, columns=distance_df.columns)
        edf_df.to_excel(output_file)
        print(f"✓ 衰减矩阵已保存到 {output_file}")
        print("衰减矩阵预览（前5x5）：")
        print(edf_df.iloc[:5, :5].round(4))
        return edf_df
    except FileNotFoundError:
        print("❌ 未找到距离矩阵文件，请先生成 '城市距离矩阵.xlsx' 或传入 distance_df 参数。")
        return None

# 如果 notebook 中已有 distance_matrix_df，则直接使用它，否则从文件读取
try:
    df_edf = compute_decay_matrix(distance_matrix_df, alpha=0.01)
except NameError:
    df_edf = compute_decay_matrix(None, alpha=0.01)


读取距离矩阵：370 城市
✓ 衰减矩阵已保存到 城市距离衰减矩阵.xlsx
衰减矩阵预览（前5x5）：
         北京市     天津市     上海市  重庆市    石家庄市
北京市   1.0000  0.4227  0.0000  0.0  0.0598
天津市   0.4227  1.0000  0.0001  0.0  0.0595
上海市   0.0000  0.0001  1.0000  0.0  0.0001
重庆市   0.0000  0.0000  0.0000  1.0  0.0000
石家庄市  0.0598  0.0595  0.0001  0.0  1.0000


# min-max normalization标准化 

产出城市距离衰减矩阵_norm.xlsx

In [8]:
import pandas as pd
import numpy as np

# Min-Max 归一化 EDF 矩阵并保存
def min_max_normalize_edf(edf_df=None, output_file="城市距离衰减矩阵_norm.xlsx"):
    try:
        if edf_df is None:
            edf_df = pd.read_excel("城市距离衰减矩阵.xlsx", index_col=0)
        print(f"读取 EDF 矩阵：{edf_df.shape[0]} 城市")
        vals = edf_df.values.astype(float)
        vmin = np.nanmin(vals)
        vmax = np.nanmax(vals)
        print(f"原始最小值={vmin:.6f}, 最大值={vmax:.6f}")
        if np.isclose(vmax, vmin):
            print("警告：矩阵所有值相同，归一化结果将全为0。")
            norm_vals = np.zeros_like(vals)
        else:
            norm_vals = (vals - vmin) / (vmax - vmin)
        norm_df = pd.DataFrame(norm_vals, index=edf_df.index, columns=edf_df.columns)
        norm_df.to_excel(output_file)
        print(f"✓ 归一化矩阵已保存到 {output_file}")
        print("归一化矩阵预览（前5x5）：")
        print(norm_df.iloc[:5, :5].round(4))
        print(f"归一化后最小={norm_vals.min():.6f}, 最大={norm_vals.max():.6f}")
        return norm_df
    except FileNotFoundError:
        print("❌ 未找到 EDF 文件 '城市距离衰减矩阵.xlsx'，请先生成。")
        return None

# 尝试使用内存变量，否则从文件读取
try:
    norm_edf_df = min_max_normalize_edf(df_edf, output_file="城市距离衰减矩阵_norm.xlsx")
except NameError:
    norm_edf_df = min_max_normalize_edf(None, output_file="城市距离衰减矩阵_norm.xlsx")


读取 EDF 矩阵：370 城市
原始最小值=0.000000, 最大值=1.000000
✓ 归一化矩阵已保存到 城市距离衰减矩阵_norm.xlsx
归一化矩阵预览（前5x5）：
         北京市     天津市     上海市  重庆市    石家庄市
北京市   1.0000  0.4227  0.0000  0.0  0.0598
天津市   0.4227  1.0000  0.0001  0.0  0.0595
上海市   0.0000  0.0001  1.0000  0.0  0.0001
重庆市   0.0000  0.0000  0.0000  1.0  0.0000
石家庄市  0.0598  0.0595  0.0001  0.0  1.0000
归一化后最小=0.000000, 最大=1.000000
